# CNN Visualization
1. Activation maps
2. Filter using gradient ascent
3. Deep dream using gradient ascent

In [1]:
%env CUDA_VISIBLE_DEVICES = ""

env: CUDA_VISIBLE_DEVICES=""


In [2]:
import os 
import tensorflow as tf 
import tensorflow.keras as keras
# tf.config.gpu.set_per_process_memory_growth(True)
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.Session()

Use a pretrained model like VGG16 and VGG19 trained on ImageNet to do the task.

In [3]:
vgg19 = keras.applications.VGG19(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

In [4]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
version = 1
export_path = './model/{}'.format(version)
builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_path)

# Build the signature_def_map.
inputs = tf.compat.v1.saved_model.build_tensor_info(vgg19.input)
add_variable = False

# l_ts = dict([(l.name, tf.compat.v1.saved_model.build_tensor_info(tf.split(tf.cast(l.output, tf.int32), [4, 15, 11], 2))) for l in vgg19.layers[1:3]])
map_dict = []
for l in vgg19.layers[1:3]:
    l_ts = tf.cast(l.output, tf.int32)
    l_ts = l_ts[:, :, :, :8]
    map_dict.append((l.name, tf.compat.v1.saved_model.build_tensor_info(l_ts)))
map_dict = dict(map_dict)


signature = (
    # method name must be CLASSIFY_METHOD_NAME, PREDICT_METHOD_NAME or REGRESS_METHOD_NAME
    tf.compat.v1.saved_model.signature_def_utils.build_signature_def(
        inputs={'input': inputs},
        outputs=map_dict,
        method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME 
    )
)


builder.add_meta_graph_and_variables(
    tf.compat.v1.keras.backend.get_session(), [tf.compat.v1.saved_model.tag_constants.SERVING],
    signature_def_map={'activation':signature,},
    strip_default_attrs=True
)
add_variable = True
builder.save()

b'./model/1/saved_model.pb'

In [7]:
!saved_model_cli show --dir ./model/1 --all

Traceback (most recent call last):
  File "/home/kjliu/.local/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/kjliu/.local/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/kjliu/.local/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.5/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.5/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last